## **4. Data Extraction & Validation & Cleansing**
- Implementasikan proses pengambilan data dari json, csv, dan pdf menjadi Data Frame Spark
- Implementasikan proses perapihan yang diperlukan terhadap masing-masing data.

#### **Persiapan Pyspark**

In [58]:
# Import Spark Session dari modul pyspark
from pyspark.sql import SparkSession

# Inisiasi & jalankan spark session
spark = SparkSession.builder.appName("UTS Praktikum Big Data").getOrCreate()

#### **Pengolahan Data 1** : *rekap-jumlah-status-desa-provinsi-kalimantan-barat-tahun-2019.csv* 

##### **CSV to Dataframe Spark**

In [59]:
data_csv = spark.read.options(delimiter=";", header=True).csv("data/rekap-jumlah-status-desa-provinsi-kalimantan-barat-tahun-2019.csv", inferSchema=True)
data_csv.show()

+----------------+-------+----+----------+----------+-----------------+
|       Kabupaten|Mandiri|Maju|Berkembang|Tertinggal|Sangat Tertinggal|
+----------------+-------+----+----------+----------+-----------------+
|   Kab Pontianak|     16|  35|         7|         2|                0|
|   Kab Kubu Raya|     14|  21|        51|        30|                1|
|      Kab Sambas|      8|  29|       108|        45|                3|
|  Kab Bengkayang|      3|   6|        63|        48|                2|
|      Kab Landak|      3|   8|        20|        99|               26|
|     Kab Sanggau|      5|  19|        66|        73|                0|
|     Kab Sekadau|      4|   6|        30|        43|                4|
|     Kab Sintang|      6|  14|       104|       180|               86|
|      Kab Melawi|      4|   3|        32|        83|               47|
|      Kab K Hulu|      8|  23|       169|        75|                3|
|    Kab Ketapang|     12|  18|        91|        96|           

#### **Pengolahan Data 2** : *daftar_desa_dan_kelurahan.json* 

##### **Akuisisi Data**

In [60]:
# merubah file json menjadi struktur data dictionary
import json
with open("data/daftar_desa_dan_kelurahan.json") as json_file:
    data_json_raw = json.load(json_file)

##### **Analisis Struktur Data**

In [61]:
for item in data_json_raw["fields"]:
    print(
        type(item), #dictionary
    )

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [62]:
for item in data_json_raw["records"]:
    print(
        type(item), #list
        item[3]
    )

<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU
<class 'list'> SELAKAU TIMUR
<class 'list'> SELAKAU TIMUR
<class 'list'> SELAKAU TIMUR
<class 'list'> SELAKAU TIMUR
<class 'list'> PEMANGKAT
<class 'list'> PEMANGKAT
<class 'list'> PEMANGKAT
<class 'list'> PEMANGKAT
<class 'list'> PEMANGKAT
<class 'list'> PEMANGKAT
<class 'list'> PEMANGKAT
<class 'list'> PEMANGKAT
<class 'list'> SEMPARUK
<class 'list'> SEMPARUK
<class 'list'> SEMPARUK
<class 'list'> SEMPARUK
<class 'list'> SEMPARUK
<class 'list'> SALATIGA
<class 'list'> SALATIGA
<class 'list'> SALATIGA
<class 'list'> SALATIGA
<class 'list'> SALATIGA
<class 'list'> TEBAS
<class 'list'> TEBAS
<class 'list'> TEBAS
<class 'list'> TEBAS
<class 'list'> TEBAS
<class 'list'> TEBAS
<class 'list'> TEBAS
<class 'list'> TEBAS
<class 'list'> TEBAS
<c

##### **Proses Cleansing**

In [63]:
my_dict = dict()
key_list = list()
result_list = list()

# inisiasi key pada dictionary
for key in data_json_raw["fields"]:
    my_dict[key["id"]] = ""
    key_list.append(key["id"])

# menyusun manual file dengan format .json
# hapus seluruh isi file
with open("data/clean_result.json", "w") as file:
    file.write("")

# tambahkan karakter [ pada awal file
with open("data/clean_result.json", "a") as file:
    file.write("[")

# inisiasi value setiap key pada dictionary
for i, value in enumerate(data_json_raw["records"]):
    for j in range(len(value)):
        my_dict[key_list[j]] = value[j]
    with open("data/clean_result.json", "a") as file:
        json.dump(my_dict, file)
        if not i+1 == len(data_json_raw["records"]): # jika ini bukan merupakan item terakhir pada dictionary
            file.write(",")

# tambahkan karakter ] pada akhir file
with open("data/clean_result.json", "a") as file:
    file.write("]") 

##### **JSON to Dataframe Spark**

In [64]:
data_json = spark.read.option("multiline", "true").json("data/clean_result.json")

#### **Pengolahan Data 3** : *lembaga-kemasyarakatan-desa.pdf* 

##### **Akuisisi Data**

In [65]:
import camelot
import tabula
import pandas as pd
data_pdf_raw = tabula.read_pdf("data/lembaga-kemasyarakatan-desa.pdf") #PDF to Dataframe Pandas

'pages' argument isn't specified.Will extract only from page 1 by default.


##### **Analisis Struktur Data**

In [66]:
print(
    type(data_pdf_raw), "\n",
    type(data_pdf_raw[0])
)

<class 'list'> 
 <class 'pandas.core.frame.DataFrame'>


##### **Proses Cleansing**

In [67]:
# menambil dataframe pada list
data_pdf = data_pdf_raw[0]

In [68]:
# menampilkan semua kolom dan baris
data_pdf 

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Jenis Lembaga Kemasyarakatan Desa (Pasal 6),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,No,Nama Kabupaten,RT,RW,PKK,Karang Taruna,Posyandu,LPM,Kelompok Tani,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,RT,Pengurus,RW,Pengurus,PKK,Pengurus,Karang Taruna,Pengurus,...,Pengurus,Kader,LPM,Pengurus,Kelompok,Pengurus,Kelompok,Pengurus,NaN,NaN
2,NaN,1,SAMBAS,588,552,214,186,196,390,206,...,73,175,"2,857",184,214,121,579,23,13.0,NaN
3,NaN,2,MEMPAWAH,932,"1,180",274,326,55,"1,175",42,...,208,766,"1,141",74,344,295,"1,581",13,35.0,NaN
4,NaN,3,SANGGAU,"1,952","1,978",82,82,157,347,5,...,65,142,"2,592",133,97,0,10,0,0.0,NaN
5,NaN,4,KETAPANG,"1,592",473,41,65,170,"1,590",169,...,516,"1,056","2,603",238,871,134,550,1,20.0,NaN
6,NaN,5,SINTANG,"1,987",526,34,34,159,446,36,...,48,111,"1,518",0,0,17,125,1,3.0,NaN
7,NaN,6,KAPUAS HULU,"1,476","1,489",0,0,265,"4,016",180,...,414,"1,232","2,161",277,"1,118",246,637,7,45.0,NaN
8,NaN,7,BENGKAYANG,79,94,0,0,5,56,4,...,21,79,"1,349",107,0,0,0,1,39.0,NaN
9,NaN,8,LANDAK,517,355,72,5,541,360,0,...,557,536,"2,558",104,0,157,273,0,0.0,NaN


In [69]:
# menyeleksi baris total keseluruhan data
data_pdf_total_keseluruhan = data_pdf.iloc[-1, 2:-1] #[row, col]
data_pdf_total_keseluruhan

Unnamed: 2                                     14,662
Jenis Lembaga Kemasyarakatan Desa (Pasal 6)    12,084
Unnamed: 3                                      1,720
Unnamed: 4                                      1,134
Unnamed: 5                                      2,077
Unnamed: 6                                     11,996
Unnamed: 7                                        914
Unnamed: 8                                      5,348
Unnamed: 9                                      2,568
Unnamed: 10                                     6,350
Unnamed: 11                                    21,079
Unnamed: 12                                     1,476
Unnamed: 13                                     3,344
Unnamed: 14                                     1,784
Unnamed: 15                                     6,685
Unnamed: 16                                        53
Unnamed: 17                                       193
Unnamed: 18                                       NaN
Name: 14, dtype: object

In [70]:
# menyeleksi seluruh isi data kecuali nomor dan total keseluruhan data
data_pdf_isi = data_pdf.iloc[2:-1, 2:-1] #[row, col]
data_pdf_isi

,Unnamed: 2,Jenis Lembaga Kemasyarakatan Desa (Pasal 6),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
2,SAMBAS,588,552,214,186,196,390,206,412,73,175,"2,857",184,214,121,579,23,13.0
3,MEMPAWAH,932,"1,180",274,326,55,"1,175",42,636,208,766,"1,141",74,344,295,"1,581",13,35.0
4,SANGGAU,"1,952","1,978",82,82,157,347,5,81,65,142,"2,592",133,97,0,10,0,0.0
5,KETAPANG,"1,592",473,41,65,170,"1,590",169,907,516,"1,056","2,603",238,871,134,550,1,20.0
6,SINTANG,"1,987",526,34,34,159,446,36,240,48,111,"1,518",0,0,17,125,1,3.0
7,KAPUAS HULU,"1,476","1,489",0,0,265,"4,016",180,"1,426",414,"1,232","2,161",277,"1,118",246,637,7,45.0
8,BENGKAYANG,79,94,0,0,5,56,4,64,21,79,"1,349",107,0,0,0,1,39.0
9,LANDAK,517,355,72,5,541,360,0,0,557,536,"2,558",104,0,157,273,0,0.0
10,SEKADAU,937,787,249,221,165,717,125,276,163,336,949,54,92,197,660,1,10.0
11,MELAWI,"1,204","1,204",33,6,169,"1,352",76,253,191,788,807,85,110,157,302,1,0.0


In [71]:
print(
    len(data_pdf_isi.columns), "\n",
    data_pdf_isi.columns
)

18 
 Index(['Unnamed: 2', 'Jenis Lembaga Kemasyarakatan Desa (Pasal 6)',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')


In [72]:
# menamai ulang kolom kolom pada dataframe
data_pdf_isi.columns = [
    "Nama Kabupaten",
    "Jumlah RT",
    "Total Pengurus RT",
    "Jumlah RW",
    "Total Pengurus RW",
    "Jumlah PKK",
    "Total Pengurus PKK",
    "Jumlah Karang Taruna",
    "Total Pengurus Karang Taruna",
    "Jumlah Posyandu",
    "Total Pengurus Posyandu",
    "Total Kader Posyandu",
    "Jumlah LPM",
    "Total Pengurus LPM",
    "Jumlah Tani",
    "Total Pengurus Tani",
    "Jumlah Sadar Wisata",
    "Total Pengurus Sadar Wisata",
]

In [73]:
data_pdf_isi = data_pdf_isi.reset_index(drop=True) # reset index pada dataframe
data_pdf_isi # menampilkan data

,Nama Kabupaten,Jumlah RT,Total Pengurus RT,Jumlah RW,Total Pengurus RW,Jumlah PKK,Total Pengurus PKK,Jumlah Karang Taruna,Total Pengurus Karang Taruna,Jumlah Posyandu,Total Pengurus Posyandu,Total Kader Posyandu,Jumlah LPM,Total Pengurus LPM,Jumlah Tani,Total Pengurus Tani,Jumlah Sadar Wisata,Total Pengurus Sadar Wisata
0,SAMBAS,588,552,214,186,196,390,206,412,73,175,"2,857",184,214,121,579,23,13.0
1,MEMPAWAH,932,"1,180",274,326,55,"1,175",42,636,208,766,"1,141",74,344,295,"1,581",13,35.0
2,SANGGAU,"1,952","1,978",82,82,157,347,5,81,65,142,"2,592",133,97,0,10,0,0.0
3,KETAPANG,"1,592",473,41,65,170,"1,590",169,907,516,"1,056","2,603",238,871,134,550,1,20.0
4,SINTANG,"1,987",526,34,34,159,446,36,240,48,111,"1,518",0,0,17,125,1,3.0
5,KAPUAS HULU,"1,476","1,489",0,0,265,"4,016",180,"1,426",414,"1,232","2,161",277,"1,118",246,637,7,45.0
6,BENGKAYANG,79,94,0,0,5,56,4,64,21,79,"1,349",107,0,0,0,1,39.0
7,LANDAK,517,355,72,5,541,360,0,0,557,536,"2,558",104,0,157,273,0,0.0
8,SEKADAU,937,787,249,221,165,717,125,276,163,336,949,54,92,197,660,1,10.0
9,MELAWI,"1,204","1,204",33,6,169,"1,352",76,253,191,788,807,85,110,157,302,1,0.0


In [74]:
# menyimpan file ekstraksi data pdf dalam bentuk csv
data_pdf_isi.to_csv("data/lembaga-kemasyarakatan-desa.csv")

##### **CSV to Dataframe Spark**

In [75]:
data_pdf_spark = spark.read.options(delimiter=",", header=True).csv("data/lembaga-kemasyarakatan-desa.csv", inferSchema=True)
data_pdf_spark.show()

+---+--------------+---------+-----------------+---------+-----------------+----------+------------------+--------------------+----------------------------+---------------+-----------------------+--------------------+----------+------------------+-----------+-------------------+-------------------+---------------------------+
|_c0|Nama Kabupaten|Jumlah RT|Total Pengurus RT|Jumlah RW|Total Pengurus RW|Jumlah PKK|Total Pengurus PKK|Jumlah Karang Taruna|Total Pengurus Karang Taruna|Jumlah Posyandu|Total Pengurus Posyandu|Total Kader Posyandu|Jumlah LPM|Total Pengurus LPM|Jumlah Tani|Total Pengurus Tani|Jumlah Sadar Wisata|Total Pengurus Sadar Wisata|
+---+--------------+---------+-----------------+---------+-----------------+----------+------------------+--------------------+----------------------------+---------------+-----------------------+--------------------+----------+------------------+-----------+-------------------+-------------------+---------------------------+
|  0|        SAM

## **5. Data Aggregation & Representation**
- Implementasikan proses kombinasi antara data set tersebut, dan lakukan statistik dasar.
- untuk menggabungkan 3 data diatas dibutuhkan sebuah kolom yang berfungsi sebagai primary key
- setelah di identifikasi didapati kolom nama kabupaten dapat dijadikan sebagai primary key

In [76]:
data_csv.select("Kabupaten").show()

+----------------+
|       Kabupaten|
+----------------+
|   Kab Pontianak|
|   Kab Kubu Raya|
|      Kab Sambas|
|  Kab Bengkayang|
|      Kab Landak|
|     Kab Sanggau|
|     Kab Sekadau|
|     Kab Sintang|
|      Kab Melawi|
|      Kab K Hulu|
|    Kab Ketapang|
|Kab Kayong Utara|
|          Jumlah|
+----------------+



In [77]:
data_json.select("Kabupaten/Kota").show()

+----------------+
|  Kabupaten/Kota|
+----------------+
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
|KABUPATEN SAMBAS|
+----------------+
only showing top 20 rows



In [78]:
data_pdf_spark.select("Nama Kabupaten").show()

+--------------+
|Nama Kabupaten|
+--------------+
|        SAMBAS|
|      MEMPAWAH|
|       SANGGAU|
|      KETAPANG|
|       SINTANG|
|   KAPUAS HULU|
|    BENGKAYANG|
|        LANDAK|
|       SEKADAU|
|        MELAWI|
|  KAYONG UTARA|
|     KUBU RAYA|
+--------------+



##### **Menyamakan Nama Format Kabupaten**

In [79]:

# Replace string column value conditionally
# https://sparkbyexamples.com/pyspark/pyspark-replace-column-values/
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
data_csv.withColumn('Kabupaten', 
    when(data_csv.Kabupaten.startswith('Kab'),regexp_replace(data_csv.Kabupaten,'Kab','')) \
   .otherwise(data_csv.Kabupaten)) \
   .show(truncate=False)

+-------------+-------+----+----------+----------+-----------------+
|Kabupaten    |Mandiri|Maju|Berkembang|Tertinggal|Sangat Tertinggal|
+-------------+-------+----+----------+----------+-----------------+
| Pontianak   |16     |35  |7         |2         |0                |
| Kubu Raya   |14     |21  |51        |30        |1                |
| Sambas      |8      |29  |108       |45        |3                |
| Bengkayang  |3      |6   |63        |48        |2                |
| Landak      |3      |8   |20        |99        |26               |
| Sanggau     |5      |19  |66        |73        |0                |
| Sekadau     |4      |6   |30        |43        |4                |
| Sintang     |6      |14  |104       |180       |86               |
| Melawi      |4      |3   |32        |83        |47               |
| K Hulu      |8      |23  |169       |75        |3                |
| Ketapang    |12     |18  |91        |96        |36               |
| Kayong Utara|4      |6   |26    

In [80]:
data_json = data_json.withColumnRenamed("Kabupaten/Kota", "Kabupaten")

In [81]:
data_json.withColumn('Kabupaten', 
    when(data_json.Kabupaten.startswith('KABUPATEN'),regexp_replace(data_json.Kabupaten,'KABUPATEN','')) \
   .otherwise(data_json.Kabupaten)) \
   .show(truncate=False)

+----------------+----------------------+---------+-------------+------------------+---------------+-----------------+---------------------------------------------------------------------------------------+---------+----------------+------+---------------+---+
|Desa/Kelurahan  |Desa/Kelurahan Terluar|Kabupaten|Kecamatan    |Ketinggian DPL (M)|Koordinat Bujur|Koordinat Lintang|Lokasi                                                                                 |Luas (Ha)|Provinsi        |Status|Tipologi       |_id|
+----------------+----------------------+---------+-------------+------------------+---------------+-----------------+---------------------------------------------------------------------------------------+---------+----------------+------+---------------+---+
|SEMELAGI BESAR  |Tidak                 | SAMBAS  |SELAKAU      |0,00              |109,047260     |1,002419         |DESA SEMELAGI BESAR KECAMATAN SELAKAU KABUPATEN SAMBAS PROVINSI KALIMANTAN BARAT       |1.572,00 |K